In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import mean_squared_error, r2_score
os.chdir('C:/Users/johan/SynologyDrive/Johan Scolaire/M2/1st semester/Machine Learning')
print(os.getcwd())

C:\Users\johan\SynologyDrive\Johan Scolaire\M2\1st semester\Machine Learning


In [37]:
def process_percentage_column(column):
    return column.str.rstrip('%').astype(float) / 100

In [39]:
#We will get the 2012,2016,2020 datasets
df_2012=pd.read_csv('Data/Features_2012.csv')
df_2016=pd.read_csv('Data/Features_2016.csv')
df_2021=pd.read_csv('Data/Features_2021.csv')

df_target_2012=pd.read_csv('Data/2012 Election Results.csv')
df_target_2016=pd.read_csv('Data/2016 Presidential Election Results by State.csv')
df_target_2020=pd.read_csv('Data/2020 elections made.csv')

df_test_2023=pd.read_csv('Data/Features_2023.csv')

df_target_2012.drop([51],axis=0,inplace=True)
df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']] = df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].apply(pd.to_numeric, errors='coerce')
df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']]=df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].div(100) #fixed

#print(df_target_2012) #nice
df_target_2020[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']]=df_target_2020[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].apply(process_percentage_column)

#Merge feature data
df_train=pd.concat([df_2012,df_2016,df_2021],ignore_index=True)
#print(df_train)

# #Merge target data
df_target=pd.concat([df_target_2012,df_target_2016,df_target_2020],ignore_index=True)
#print(df_target)
#Make indices nice etc
df_train.set_index("NAME",inplace=True) #nice
df_target.set_index("STATE",inplace=True)
df_test_2023.set_index("NAME",inplace=True)
#print(df_train)

#Fixes:
#GRAPI is wrong for anything other than 2012. RIP
df_train.drop(df_train.columns[df_train.columns.str.contains('GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME')], axis=1, inplace=True)
df_test_2023.drop(df_test_2023.columns[df_test_2023.columns.str.contains('GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME')], axis=1, inplace=True)

#df_train
df_test_2023

MEDIAN AGE  % under 18  % 65+  % 18+  % One race WHITE  \
NAME                                                                           
Alabama                     39.6        22.1   18.2   77.9              64.7   
Alaska                      36.5        23.8   14.0   76.2              59.6   
Arizona                     39.3        21.3   19.3   78.7              58.3   
Arkansas                    38.9        22.9   18.0   77.1              68.9   
California                  38.2        21.7   16.2   78.3              38.5   
Colorado                    37.9        20.6   16.1   79.4              70.4   
Connecticut                 41.5        20.0   19.1   80.0              64.5   
Delaware                    42.0        20.5   21.3   79.5              59.3   
District of Columbia        34.9        18.6   13.1   81.4              38.8   
Florida                     42.8        19.4   21.7   80.6              55.5   
Georgia                     37.9        23.0   15.4   77.0              50.3   
Hawaii                      41.4        20.5   21.1   79.5              21.9   
Idaho                       37.8        23.7   17.4   76.3              81.7   
Illinois                    39.5        21.5   17.6   78.5              60.7   
Indiana                     38.3        23.0   17.2   77.0              76.7   
Iowa                        39.1        22.6   18.5   77.4              84.2   
Kansas                      37.9        23.4   17.6   76.6              75.9   
Kentucky                    39.2        22.4   17.8   77.6              82.5   
Louisiana                   38.7        23.3   17.4   76.7              56.7   
Maine                       44.9        17.6   22.9   82.4              90.1   
Maryland                    39.8        22.0   17.3   78.0              47.9   
Massachusetts               40.3        19.2   18.5   80.8              67.9   
Michigan                    40.5        21.0   19.3   79.0              73.8   
Minnesota                   39.1        22.6   17.9   77.4              76.7   
Mississippi                 39.3        23.0   17.6   77.0              55.6   
Missouri                    39.3        22.1   18.4   77.9              77.8   
Montana                     40.6        20.8   20.5   79.2              84.6   
Nebraska                    37.4        24.2   17.2   75.8              77.7   
Nevada                      39.4        21.4   17.4   78.6              49.8   
New Hampshire               43.4        17.9   20.7   82.1              87.5   
New Jersey                  40.4        21.6   17.7   78.4              53.5   
New Mexico                  39.9        21.2   19.9   78.8              47.5   
New York                    40.2        20.2   18.6   79.8              55.1   
North Carolina              39.4        21.4   17.7   78.6              61.4   
North Dakota                36.3        22.9   17.1   77.1              82.5   
Ohio                        39.9        21.8   18.8   78.2              76.6   
Oklahoma                    37.1        23.8   16.7   76.2              64.6   
Oregon                      40.7        19.6   19.6   80.4              73.9   
Pennsylvania                41.1        20.2   20.0   79.8              74.0   
Rhode Island                40.8        18.5   19.4   81.5              69.7   
South Carolina              40.5        21.3   19.3   78.7              63.6   
South Dakota                38.5        23.8   18.4   76.2              80.5   
Tennessee                   39.1        21.9   17.5   78.1              72.3   
Texas                       35.9        24.7   13.8   75.3              47.7   
Utah                        32.3        27.3   12.2   72.7              78.6   
Vermont                     43.7        17.7   22.2   82.3              89.9   
Virginia                    39.3        21.5   17.2   78.5              59.8   
Washington                  38.6        21.1   17.1   78.9              65.2   
West Virginia         

In [41]:
#XGBoost Regression
X=df_train
y=df_target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
estimators = [
    ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ('clf', XGBRegressor(random_state=8))
]
pipe=Pipeline(steps=estimators)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
print(y_pred)
test_r2=r2_score(y_test,y_pred)
print(test_r2)
df_results=pd.DataFrame(y_pred)
df_results.to_csv(f'Data\Results_2023.csv', index=True)

<>:17: SyntaxWarning: invalid escape sequence '\R'
<>:17: SyntaxWarning: invalid escape sequence '\R'
C:\Users\johan\AppData\Local\Temp\ipykernel_4548\93061831.py:17: SyntaxWarning: invalid escape sequence '\R'
  df_results.to_csv(f'Data\Results_2023.csv', index=True)


[[ 5.8764237e-01  4.2208457e-01 -4.4479311e-04]
 [ 6.6405147e-01  2.9864162e-01  5.3207636e-02]
 [ 5.1026726e-01  4.2447793e-01  2.4289617e-02]
 [ 4.2979029e-01  4.6161908e-01  7.7213976e-05]
 [ 6.1258322e-01  4.4474581e-01  2.0710150e-02]
 [ 3.7948176e-01  5.1210403e-01  3.3809431e-02]
 [ 3.4079510e-01  5.5974567e-01  3.3519816e-02]
 [ 4.9558833e-01  4.3185139e-01  4.2652991e-02]
 [ 4.7316393e-01  4.5223954e-01  2.4412265e-02]
 [ 6.0768116e-01  3.9890397e-01  2.3214661e-02]
 [ 5.0349164e-01  4.5410722e-01  1.3812804e-03]
 [ 6.0486197e-01  3.8291955e-01  1.4580834e-02]
 [ 4.6138668e-01  4.9556231e-01  3.5527755e-02]
 [ 4.1797444e-01  5.9514713e-01  4.1693050e-02]
 [ 5.5022734e-01  4.6613246e-01  8.4952917e-05]
 [ 4.7160029e-01  4.4925895e-01  2.6956921e-02]
 [ 3.5453191e-01  5.1257610e-01  4.1193441e-02]
 [ 5.4365855e-01  4.2815158e-01  1.4883623e-02]
 [ 5.6567830e-01  3.8201475e-01  3.3742640e-02]
 [ 4.7854140e-01  3.6013213e-01  4.0924989e-02]
 [ 3.7357980e-01  5.6302679e-01 -4.56653

In [27]:
X_test


MEDIAN AGE  % under 18  % 65+  % 18+  % One race WHITE  \
NAME                                                                    
New Hampshire        41.9        74.5   51.3   14.7              94.0   
Hawaii               40.2        21.1   19.6   78.9              22.2   
New Hampshire        43.1        18.4   19.3   81.6              87.9   
California           35.5        71.2   50.7   12.1              62.1   
Texas                35.5        25.3   13.2   74.7              47.9   
Oklahoma             37.2        24.0   16.2   76.0              65.6   
Indiana              37.6        71.9   51.4   15.0              83.5   
Colorado             36.7        73.2   50.0   13.4              84.0   
Vermont              42.9        18.1   20.6   81.9              91.1   
Massachusetts        39.9        19.5   17.4   80.5              69.2   
Minnesota            37.6        72.2   50.8   13.6              85.5   
New York             39.8        20.7   17.5   79.3              55.6   
Virginia             38.2        73.5   51.4   14.6              67.8   
Louisiana            36.5        71.9   51.8   14.4              62.0   
New Jersey           39.4        73.2   51.9   14.1              68.9   
Nebraska             37.2        24.6   16.4   75.4              78.5   
Montana              40.1        74.0   50.0   17.6              89.0   
Pennsylvania         40.9        20.6   19.0   79.4              74.9   
Connecticut          40.9        74.6   51.8   16.1              76.7   
New Hampshire        42.7        76.2   50.7   17.0              93.4   
Wyoming              36.9        71.7   49.4   13.0              90.9   
Nebraska             36.3        70.7   50.8   13.8              88.1   
Alaska               33.8        70.1   47.8    8.5              66.5   
Kansas               37.3        23.9   16.7   76.1              76.0   
Tennessee            39.2        22.0   17.0   78.0              73.1   
Rhode Island         40.2        75.2   52.2   16.5              80.5   
Wisconsin            38.8        72.6   50.8   14.4              86.7   
Alabama              38.2        72.2   52.4   14.5              69.1   
Utah                 29.9        64.2   50.2    9.5              88.1   
Arkansas             37.6        71.5   51.7   15.0              78.3   
Kentucky             38.4        72.5   51.4   14.0              87.8   

               % One race BLACK  % Two or more races  % One race Mexican  \
NAME                                                                       
New Hampshire               1.3                  1.8                 0.9   
Hawaii                      2.0                  5.1                20.8   
New Hampshire               1.5                  0.8                87.3   
California                  6.0                  4.4                31.8   
Texas                      11.9                  0.6                39.4   
Oklahoma                    6.9                  0.6                62.8   
Indiana                     9.3                  2.4                 5.0   
Colorado                    4.2                  3.3                15.4   
Vermont                     1.1                  0.6                90.6   
Massachusetts               6.7                  0.9                67.5   
Minnesota                   5.3                  2.5                 3.5   
New York                   14.2                  0.7                53.5   
Virginia                   19.0                  3.7                 2.0   
Louisiana                  32.4                  2.0                 2.0   
New Jersey                 13.6                  2.5                 2.5   
Nebraska                    4.7                  0.8                76.4   
Montana                     0.4                  3.1                 2.6   
Pennsylvania               10.4                  0.6                73.6   
Connecticut                10.6                  3.1                 1.7   
New Hamps

In [21]:
len(df_results)

31

In [11]:
search_space = {
    'clf__max_depth': Integer(2,8),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}
                                       #cv: number of folds
opt = BayesSearchCV(pipe, search_space, cv=3, n_iter=5, scoring='r2', random_state=8) 
# in reality, you may consider setting cv and n_iter to higher values

In [13]:
opt.fit(X_train, y_train)
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
# Output results

print("Best hyperparameters:", opt.best_params_)
print("R^2 score on the test set:", test_r2)

Best hyperparameters: OrderedDict({'clf__colsample_bylevel': 0.7750018497221565, 'clf__colsample_bynode': 0.5614437441596264, 'clf__colsample_bytree': 0.9126202065825759, 'clf__gamma': 8.289497472648083, 'clf__learning_rate': 0.4299244814327041, 'clf__max_depth': 6, 'clf__reg_alpha': 2.784887532399771, 'clf__reg_lambda': 1.67027558902639, 'clf__subsample': 0.5966102807384807})
R^2 score on the test set: -1.4675713777542114


In [59]:
y_2023 = pipe.predict(df_test_2023)
print(y_2023)
# test_r2=r2_score(y_test,y_pred)
# print(test_r2)
df_results=pd.DataFrame(y_2023)
df_results.to_csv(f'Data\Results_2023.csv', index=True)

[[0.3743298  0.58052945 0.03836443]
 [0.39247748 0.5300566  0.04339309]
 [0.51902586 0.4355908  0.03772557]
 [0.39179045 0.5637133  0.03731529]
 [0.6546774  0.29105216 0.03596279]
 [0.5042735  0.39189276 0.03750478]
 [0.5796125  0.39290798 0.04194616]
 [0.5237591  0.37847286 0.03736698]
 [0.78405476 0.06769337 0.03674998]
 [0.5020563  0.43539864 0.0366193 ]
 [0.49549797 0.40663767 0.03742824]
 [0.6461086  0.2972873  0.03717332]
 [0.3578605  0.5077965  0.04148975]
 [0.58859617 0.40390128 0.03652459]
 [0.43445987 0.44463697 0.03596126]
 [0.46184215 0.4401934  0.03422032]
 [0.37209854 0.5278568  0.04016501]
 [0.4440545  0.4781175  0.04313305]
 [0.42888045 0.45057875 0.04308501]
 [0.4342084  0.40975422 0.04332659]
 [0.67913336 0.34698153 0.04227941]
 [0.5888449  0.38760495 0.04630915]
 [0.502799   0.43916595 0.03627715]
 [0.50083256 0.41943723 0.03873556]
 [0.37756544 0.59536517 0.04173687]
 [0.36529326 0.44695652 0.0381314 ]
 [0.38032228 0.545015   0.03685782]
 [0.47002873 0.39716625 0.04

<>:6: SyntaxWarning: invalid escape sequence '\R'
<>:6: SyntaxWarning: invalid escape sequence '\R'
C:\Users\johan\AppData\Local\Temp\ipykernel_4548\99628672.py:6: SyntaxWarning: invalid escape sequence '\R'
  df_results.to_csv(f'Data\Results_2023.csv', index=True)


In [121]:
import seaborn as sns
import geopandas as gpd

from PIL import Image
from matplotlib.patches import Patch, Circle

edge_color = "#30011E"
background_color = "#fafafa"

sns.set_style({
    "font.family": "serif",
    "figure.facecolor": background_color,
    "axes.facecolor": background_color,
})

In [123]:
def translate_geometries(df, x, y, scale, rotate):
    df.loc[:, "geometry"] = df.geometry.translate(yoff=y, xoff=x)
    center = df.dissolve().centroid.iloc[0]
    df.loc[:, "geometry"] = df.geometry.scale(xfact=scale, yfact=scale, origin=center)
    df.loc[:, "geometry"] = df.geometry.rotate(rotate, origin=center)
    return df

def adjust_maps(df):
    df_main_land = df[~df.STATEFP.isin(["02", "15"])]
    df_alaska = df[df.STATEFP == "02"]
    df_hawaii = df[df.STATEFP == "15"]

    df_alaska = translate_geometries(df_alaska, 1300000, -4900000, 0.5, 32)
    df_hawaii = translate_geometries(df_hawaii, 5400000, -1500000, 1, 24)

    return pd.concat([df_main_land, df_alaska, df_hawaii])

In [125]:
# counties = gpd.read_file("./data/cb_2018_us_county_500k/")
# counties = counties[~counties.STATEFP.isin(["72", "69", "60", "66", "78"])]
# counties = counties.set_index("GEOID")

states = gpd.read_file("./data_drawing/cb_2018_us_state_500k/")
states = states[~states.STATEFP.isin(["72", "69", "60", "66", "78"])]

# counties = counties.to_crs("ESRI:102003")
states = states.to_crs("ESRI:102003")

# counties = adjust_maps(counties)
states = adjust_maps(states)

In [127]:
def create_state_colors(results):
    colors = ["blue", "red", "yellow"]
    state_colors = np.array([colors[np.argmax(results[i])] for i in range(len(results))])
    return state_colors



In [129]:
states = states.set_index("STATEFP")
states = states.sort_index()

In [131]:
states

STATENS     AFFGEOID GEOID STUSPS                  NAME LSAD  \
STATEFP                                                                  
01       01779775  0400000US01    01     AL               Alabama   00   
02       01785533  0400000US02    02     AK                Alaska   00   
04       01779777  0400000US04    04     AZ               Arizona   00   
05       00068085  0400000US05    05     AR              Arkansas   00   
06       01779778  0400000US06    06     CA            California   00   
08       01779779  0400000US08    08     CO              Colorado   00   
09       01779780  0400000US09    09     CT           Connecticut   00   
10       01779781  0400000US10    10     DE              Delaware   00   
11       01702382  0400000US11    11     DC  District of Columbia   00   
12       00294478  0400000US12    12     FL               Florida   00   
13       01705317  0400000US13    13     GA               Georgia   00   
15       01779782  0400000US15    15     HI                Hawaii   00   
16       01779783  0400000US16    16     ID                 Idaho   00   
17       01779784  0400000US17    17     IL              Illinois   00   
18       00448508  0400000US18    18     IN               Indiana   00   
19       01779785  0400000US19    19     IA                  Iowa   00   
20       00481813  0400000US20    20     KS                Kansas   00   
21       01779786  0400000US21    21     KY              Kentucky   00   
22       01629543  0400000US22    22     LA             Louisiana   00   
23       01779787  0400000US23    23     ME                 Maine   00   
24       01714934  0400000US24    24     MD              Maryland   00   
25       00606926  0400000US25    25     MA         Massachusetts   00   
26       01779789  0400000US26    26     MI              Michigan   00   
27       00662849  0400000US27    27     MN             Minnesota   00   
28       01779790  0400000US28    28     MS           Mississippi   00   
29       01779791  0400000US29    29     MO              Missouri   00   
30       00767982  0400000US30    30     MT               Montana   00   
31       01779792  0400000US31    31     NE              Nebraska   00   
32       01779793  0400000US32    32     NV                Nevada   00   
33       01779794  0400000US33    33     NH         New Hampshire   00   
34       01779795  0400000US34    34     NJ            New Jersey   00   
35       00897535  0400000US35    35     NM            New Mexico   00   
36       01779796  0400000US36    36     NY              New York   00   
37       01027616  0400000US37    37     NC        North Carolina   00   
38       01779797  0400000US38    38     ND          North Dakota   00   
39       01085497  0400000US39    39     OH                  Ohio   00   
40       01102857  0400000US40    40     OK              Oklahoma   00   
41       01155107  0400000US41    41     OR                Oregon   00   
42       01779798  0400000US42    42     PA          Pennsylvania   00   
44       01219835  0400000US44    44     RI          Rhode Island   00   
45       01779799  0400000US45    45     SC        South Carolina   00   
46       01785534  0400000US46    46     SD          South Dakota   00   
47       01325873  0400000US47    47     TN             Tennessee   00   
48       01779801  0400000US48    48     TX                 Texas   00   
49       01455989  0400000US49    49     UT                  Utah   00   
50       01779802  0400000US50    50     VT               Vermont   00   
51       01779803  0400000US51    51     VA              Virginia   00   
53       01779804  0400000US53    53     WA            Washington   00   
54       01779805  0400000US54    54     WV         West Virginia   00   
55       01779806  0400000US55    55     WI             Wisconsin   00   
56       01779807  0400000US56    56     WY               Wyoming   00   

                 ALAND        AWATER  \
STATEFP                                
01     

In [133]:
ax = states.plot(edgecolor=edge_color, color=create_state_colors(y_2023), linewidth=0.5)

plt.axis("off")
plt.show()